In [1]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns
import os
import scipy
import scipy.io

In [2]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [3]:
a = scipy.io.loadmat("../data/zeisel/zeisel_data.mat")
data= a['zeisel_data'].T
N,d=data.shape

#load labels (first level of the hierarchy) from file
a = scipy.io.loadmat("../data/zeisel/zeisel_labels1.mat")
l_aux = a['zeisel_labels1']
l_0=[l_aux[i][0] for i in range(l_aux.shape[0])]
#load labels (second level of the hierarchy) from file
a = scipy.io.loadmat("../data/zeisel/zeisel_labels2.mat")
l_aux = a['zeisel_labels2']
l_1=[l_aux[i][0] for i in range(l_aux.shape[0])]
#construct an array with hierarchy labels
labels=np.array([l_0, l_1])

# load names from file 
a = scipy.io.loadmat("../data/zeisel/zeisel_names.mat")
names0=np.array([a['zeisel_names'][i][0][0] for i in range(N)])
names1=[a['zeisel_names'][i][1][0] for i in range(N)]

np.random.seed(100)
slices = np.random.permutation(np.arange(data.shape[0]))
upto = int(.8 * len(data))

train_data = data[slices[:upto]]
test_data = data[slices[upto:]]

train_labels = names0[slices[:upto]]
test_labels = names0[slices[upto:]]


scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

train_data = Tensor(train_data).to(device)
test_data = Tensor(test_data).to(device)

In [4]:
N = 10000
z_size = 100

# really good results for vanilla VAE on synthetic data with EPOCHS set to 50, 
# but when running locally set to 10 for reasonable run times
n_epochs = 600
batch_size = 64
lr = 0.000005
b1 = 0.9
b2 = 0.999

global_t = 4
k = 50

In [5]:
def train_model(train_data, model):
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=lr, 
                                 betas = (b1,b2))
        
    for epoch in range(1, n_epochs+1):
        train(train_data, 
              model, 
              optimizer, 
              epoch, 
              batch_size)
        model.t = max(0.001, model.t * 0.99)

        
    return model

def save_model(base_path, model):
    # make directory
    if not os.path.exists(os.path.dirname(base_path)):
        try:
            os.makedirs(os.path.dirname(base_path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise Exception("COULD NOT MAKE PATH")
    with open(base_path, 'wb') as PATH:
        torch.save(model.state_dict(), PATH)
        
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, a = 0.01)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

In [6]:
def top_logits_gumbel_globalgate_vae(data, model):
    assert isinstance(model, VAE_Gumbel_GlobalGate)
    with torch.no_grad():
        w = model.logit_enc.clone().view(-1)
        top_k_logits = torch.topk(w, k = model.k, sorted = True)[1]
        enc_top_logits = torch.nn.functional.one_hot(top_k_logits, num_classes = data.shape[1]).sum(dim = 0)
        
        #subsets = sample_subset(w, model.k,model.t,True)
        subsets = sample_subset(w, model.k,model.t)
        #max_idx = torch.argmax(subsets, 1, keepdim=True)
        #one_hot = Tensor(subsets.shape)
        #one_hot.zero_()
        #one_hot.scatter_(1, max_idx, 1)

        
    return enc_top_logits, subsets

def top_logits_gumbel_runningstate_vae(data, model):
    assert isinstance(model, VAE_Gumbel_RunningState)
    with torch.no_grad():
        w = model.logit_enc.clone().view(-1)
        top_k_logits = torch.topk(w, k = model.k, sorted = True)[1]
        enc_top_logits = torch.nn.functional.one_hot(top_k_logits, num_classes = data.shape[1]).sum(dim = 0)
        
        #subsets = sample_subset(w, model.k,model.t,True)
        subsets = sample_subset(w, model.k,model.t)
        #max_idx = torch.argmax(subsets, 1, keepdim=True)
        #one_hot = Tensor(subsets.shape)
        #one_hot.zero_()
        #one_hot.scatter_(1, max_idx, 1)

        
    return enc_top_logits, subsets

def top_logits_gumbel_concrete_vae_nsml(data, model):
    assert isinstance(model, ConcreteVAE_NMSL)
    
    with torch.no_grad():

        w = gumbel_keys(model.logit_enc, EPSILON = torch.finfo(torch.float32).eps)
        w = torch.softmax(w/model.t, dim = -1)
        subset_indices = w.clone().detach()

        #max_idx = torch.argmax(subset_indices, 1, keepdim=True)
        #one_hot = Tensor(subset_indices.shape)
        #one_hot.zero_()
        #one_hot.scatter_(1, max_idx, 1)

        all_subsets = subset_indices.sum(dim = 0)

        inds = torch.argsort(subset_indices.sum(dim = 0), descending = True)[:model.k]
        all_logits = torch.nn.functional.one_hot(inds, num_classes = data.shape[1]).sum(dim = 0)
        
        
        
        
    return all_logits, all_subsets

In [7]:
model = VAE_Gumbel_RunningState(train_data.shape[1], 200, 50, k = k, t = global_t, alpha = 0.9, bias = False)


In [8]:
model

VAE_Gumbel_RunningState(
  (encoder): Sequential(
    (0): Linear(in_features=4000, out_features=200, bias=False)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (enc_mean): Linear(in_features=200, out_features=50, bias=False)
  (enc_logvar): Linear(in_features=200, out_features=50, bias=False)
  (decoder): Sequential(
    (0): Linear(in_features=50, out_features=200, bias=False)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=200, out_features=4000, bias=False)
    (3): Sigmoid()
  )
  (weight_creator): Sequential(
    (0): Linear(in_features=4000, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=200, out_features=4000, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
)

In [ ]:
model.to(device)
model.apply(weights_init)
train_model(train_data, model)
model.set_burned_in()

Train Epoch: 1 [0/2404 (0%)]	Loss: 3601.575439
Train Epoch: 1 [1280/2404 (53%)]	Loss: 3587.273682
====> Epoch: 1 Average loss: 3584.1322
Train Epoch: 2 [0/2404 (0%)]	Loss: 3544.147217
Train Epoch: 2 [1280/2404 (53%)]	Loss: 3558.588135
====> Epoch: 2 Average loss: 3557.5481
Train Epoch: 3 [0/2404 (0%)]	Loss: 3493.164307
Train Epoch: 3 [1280/2404 (53%)]	Loss: 3550.557861
====> Epoch: 3 Average loss: 3532.8485
Train Epoch: 4 [0/2404 (0%)]	Loss: 3493.610596
Train Epoch: 4 [1280/2404 (53%)]	Loss: 3518.282715
====> Epoch: 4 Average loss: 3511.2554
Train Epoch: 5 [0/2404 (0%)]	Loss: 3517.474609
Train Epoch: 5 [1280/2404 (53%)]	Loss: 3495.145996
====> Epoch: 5 Average loss: 3490.6198
Train Epoch: 6 [0/2404 (0%)]	Loss: 3430.353027
Train Epoch: 6 [1280/2404 (53%)]	Loss: 3490.442383
====> Epoch: 6 Average loss: 3461.2573
Train Epoch: 7 [0/2404 (0%)]	Loss: 3440.864258
Train Epoch: 7 [1280/2404 (53%)]	Loss: 3439.857666
====> Epoch: 7 Average loss: 3448.9637
Train Epoch: 8 [0/2404 (0%)]	Loss: 3438.1

In [ ]:
test_data

In [ ]:
model(test_data)[0]

In [ ]:
top_logits_running_state = top_logits_gumbel_runningstate_vae(test_data, model)

In [ ]:
torch.argsort(top_logits_running_state[0], descending = True)[:k]

In [ ]:
inds_running_state = torch.argsort(top_logits_running_state[1], descending = True)[:50].cpu().numpy()

In [ ]:
len(labels[0])
print("HOW TO GET NAME OF FEATURES?")

In [ ]:
#save_model("../data/models/final_run_zeisel/runningstate_vae/k_50/model.pt", model)

Train Global Gate too.

In [ ]:
model = VAE_Gumbel_GlobalGate(train_data.shape[1], 200, 50, k = k, t = global_t, bias = False)
model.to(device)
model.apply(weights_init)
train_model(train_data, model)

In [ ]:
top_logits_global_gate = top_logits_gumbel_globalgate_vae(test_data, model)

In [ ]:
inds_global_gate = torch.argsort(top_logits_global_gate[1], descending = True)[:50].cpu().numpy()

In [ ]:
#save_model("../data/models/final_run_zeisel/global_gate/k_50/model.pt", model)

Train Concrete

In [ ]:
model = ConcreteVAE_NMSL(train_data.shape[1], 200, 50, k = k, t = global_t, bias = False)
model.to(device)
model.apply(weights_init)
train_model(train_data, model)

In [ ]:
top_logits_concrete = top_logits_gumbel_concrete_vae_nsml(test_data, model)

In [ ]:
torch.argsort(top_logits_concrete[0], descending = True)[:k]

In [ ]:
inds_concrete_vae = torch.argsort(top_logits_concrete[1], descending = True)[:50].cpu().numpy()

In [ ]:
#save_model("../data/models/final_run_zeisel/concrete_vae/k_50/model.pt", model)

Compare models and visualize.

In [ ]:
inds_running_state

In [ ]:
inds_concrete_vae

In [ ]:
np.intersect1d(inds_running_state, inds_concrete_vae)

### Embedding for actual data

In [ ]:
import umap
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE

In [ ]:
? plt.colorbar

In [ ]:
def graph_zeisel(data, labels, model_name, zeisel_encoder):
    # seems good too
    #embedding = umap.UMAP(n_neighbors=5).fit_transform(data)
    
    # seems the best
    embedding = umap.UMAP(n_neighbors=10, min_dist= 0.05).fit_transform(data)
    
    # seems good
    #embedding = TSNE(n_components=2).fit_transform(data)
    
    # weird
    #embedding = TSNE(n_components=2, perplexity=35).fit_transform(data)
    
    fig, ax = plt.subplots(1, figsize=(12, 8.5))
    
    plt.scatter(*embedding.T, c = zeisel_encoder.transform(labels))
    plt.setp(ax, xticks=[], yticks=[])
    
    cbar = plt.colorbar(ticks=np.arange(8), boundaries = np.arange(8) - 0.5)
    cbar.ax.set_yticklabels(zeisel_encoder.classes_)
    
    plt.title(f"Clustering Zeisel Data When Using {model_name} Model")

In [ ]:
zeisel_encoder = LabelEncoder()
zeisel_encoder.fit(y = train_labels)

In [ ]:
graph_zeisel(test_data.cpu().numpy(), test_labels, "No", zeisel_encoder)

In [ ]:
graph_zeisel(test_data[:, inds_global_gate].cpu().numpy(), test_labels, "Global Gate VAE", zeisel_encoder)

In [ ]:
graph_zeisel(test_data[:, inds_running_state].cpu().numpy(), test_labels, "Running State VAE", zeisel_encoder)

In [ ]:
graph_zeisel(test_data[:, inds_concrete_vae].cpu().numpy(), test_labels, "Concrete VAE", zeisel_encoder)